In [80]:
import pyvisa
import pandas as pd
import numpy as np

In [81]:
rm = pyvisa.ResourceManager()
resources = rm.list_resources()
resource = resources[0]
spectrum_analyzer_resource = rm.open_resource(resource)

In [82]:
class SpectrumAnalyzer:
    def __init__(self, spectrum_analyzer_resource):
        self.spectrum_analyzer_resource = spectrum_analyzer_resource

    def get_id(self) -> str:
        return self.format_response(self.spectrum_analyzer_resource.query('*IDN?'))
    
    def set_center_frequency(self, frequency : float):
        self.ensure_range(0, 7.5e9, frequency)
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:CENTer {frequency}')
    
    def get_center_frequency(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SENSe:FREQuency:CENTer?')))
    
    def set_start_frequency(self, frequency : float):
        self.ensure_range(0, 7.5e9, frequency)
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:STARt {frequency}')
    
    def get_start_frequency(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SENSe:FREQuency:STARt?')))
    
    def set_stop_frequency(self, frequency : float):
        self.ensure_range(0, 7.5e9, frequency)
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:STOP {frequency}')

    def get_stop_frequency(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SENSe:FREQuency:STOP?')))
    
    def set_span(self, span : float):
        self.ensure_range(0, 7.5e9, span)
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:SPAN {span}')
    
    def set_full_span(self):
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:SPAN:FULL')

    def get_span(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SENSe:FREQuency:SPAN?')))

    def enable_tracking_generator(self):
        self.spectrum_analyzer_resource.write('OUTPut:STATe ON')

    def disable_tracking_generator(self):
        self.spectrum_analyzer_resource.write('OUTPut:STATe OFF')

    def is_tracking_generator_enable(self)-> bool:
        return self.format_response(self.spectrum_analyzer_resource.query('OUTPut:STATe?')) == '1'
    
    def set_tracking_generator_offset(self, level: int):
        self.ensure_range(-200, 200, level)
        self.spectrum_analyzer_resource.write(f'SOURce:CORRection:OFFSet {level}')

    def get_tracking_generator_offset(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SOURce:CORRection:OFFSet?')))
    
    def set_tracking_generator_level(self, level: int):
        self.ensure_range(-40, 0, level)
        self.spectrum_analyzer_resource.write(f'SOURce:POWer:LEVel:IMMediate:AMPLitude {level}')

    def get_tracking_generator_level(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SOURce:POWer:LEVel:IMMediate:AMPLitude?')))

    def get_harmonics_frequency(self) -> np.array:
        return self.format_response(self.spectrum_analyzer_resource.query(':READ:HARMonics:FREQuency:ALL?'))
    
    def get_frequency_power(self, frequency) -> np.array:
        self.set_center_frequency(frequency)
        x = self.spectrum_analyzer_resource.query("TRACe:DATA? TRACE1")
        x = x.split(" ", 1)[-1]
        x = x.split(",")
        x = [float(a) for a in x]
        x = np.array(x)
        return np.median(x)
    
    def set_sweep_points(self, points: int):
        self.ensure_range(101, 3001, points)
        self.spectrum_analyzer_resource.write(f':SENSe:SWEep:POINts {points}')

    def get_sweep_points(self) -> int:
        return int(self.format_response(self.spectrum_analyzer_resource.query(':SENSe:SWEep:POINts?')))
    
    def get_trace(self):
        x = self.spectrum_analyzer_resource.query("TRACe:DATA? TRACE1")
        x = x.split(" ", 1)[-1]
        x = x.split(",")
        x = [float(a) for a in x]
        x = np.array(x)
        return x
    
    def get_trace_frequencies(self):
        start_frequecy = self.get_start_frequency()
        stop_frequency = self.get_stop_frequency()
        sweep_points = self.get_sweep_points()
        return np.linspace(start_frequecy, stop_frequency, sweep_points)
    
    def ensure_range(self, min, max, value):
        if value < min and value > max:
            raise ValueError(f'Value must be between {min} and {max}')
    
    def format_response(self, response):
        return response.strip()


In [84]:
spectrum_analyzer = SpectrumAnalyzer(spectrum_analyzer_resource)

In [85]:
spectrum_analyzer.set_sweep_points(3001)

In [86]:
spectrum_analyzer.set_full_span()
print(f"{spectrum_analyzer.get_start_frequency()} {spectrum_analyzer.get_stop_frequency()}")

100000.0 3200000000.0


In [87]:
spectrum_analyzer.enable_tracking_generator()

In [90]:
df_total = pd.DataFrame(columns=['frequency', 'gain', 'plant', 'distance'])

In [110]:
trace = spectrum_analyzer.get_trace()
frequencies = spectrum_analyzer.get_trace_frequencies()

data = {
    "frequency": frequencies,
    "gain": trace,
    "plant": "aloevera",
    "distance": 0
}

In [111]:
df = pd.DataFrame(data=data)
df

,frequency,gain,plant,distance
0,1.000000e+05,-18.16861,aloevera,0
1,1.166633e+06,-24.20336,aloevera,0
2,2.233267e+06,-30.23811,aloevera,0
3,3.299900e+06,-36.27286,aloevera,0
4,4.366533e+06,-42.30761,aloevera,0
...,...,...,...,...
2996,3.195733e+09,-63.70330,aloevera,0
2997,3.196800e+09,-63.83168,aloevera,0
2998,3.197867e+09,-63.96006,aloevera,0
2999,3.198933e+09,-64.08843,aloevera,0


In [112]:
df_total = pd.concat([df_total, df])
df_total

,frequency,gain,plant,distance
0,1.000000e+05,-17.63676,aloevera,20
1,1.166633e+06,-27.17507,aloevera,20
2,2.233267e+06,-36.71338,aloevera,20
3,3.299900e+06,-46.25169,aloevera,20
4,4.366533e+06,-55.78999,aloevera,20
...,...,...,...,...
2996,3.195733e+09,-63.70330,aloevera,0
2997,3.196800e+09,-63.83168,aloevera,0
2998,3.197867e+09,-63.96006,aloevera,0
2999,3.198933e+09,-64.08843,aloevera,0


In [ ]:
import plotly.graph_objects as go

# Exemple de sous-ensembles de données
df_aloe20 = df_total[np.logical_and(df_total['distance'] == 20, df_total['plant'] == 'aloevera')]
df_aloe15 = df_total[np.logical_and(df_total['distance'] == 15, df_total['plant'] == 'aloevera')]
df_aloe10 = df_total[np.logical_and(df_total['distance'] == 10, df_total['plant'] == 'aloevera')]
df_aloe5 = df_total[np.logical_and(df_total['distance'] == 5, df_total['plant'] == 'aloevera')]
df_aloe_2_branch = df_total[np.logical_and(df_total['distance'] == 0, df_total['plant'] == 'aloevera')]

# Création d'une figure Plotly
fig = go.Figure()

# Ajout de courbes pour chaque catégorie
fig.add_trace(go.Scatter(
    x=df_aloe20['frequency'], 
    y=df_aloe20['gain'], 
    mode='lines',
    name='Aloevera 20cm'
))
fig.add_trace(go.Scatter(
    x=df_aloe15['frequency'], 
    y=df_aloe15['gain'], 
    mode='lines',
    name='Aloevera 15cm'
))
fig.add_trace(go.Scatter(
    x=df_aloe10['frequency'], 
    y=df_aloe10['gain'], 
    mode='lines',
    name='Aloevera 10cm'
))
fig.add_trace(go.Scatter(
    x=df_aloe5['frequency'], 
    y=df_aloe5['gain'], 
    mode='lines',
    name='Aloevera 5cm'
))
fig.add_trace(go.Scatter(
    x=df_aloe_2_branch['frequency'], 
    y=df_aloe_2_branch['gain'], 
    mode='lines',
    name='Aloevera 2 branches'
))

# Mise en page personnalisée
fig.update_layout(
    title="Frequency vs Gain",
    xaxis_title="Frequency (Hz)",
    yaxis_title="Gain (dB)",
    legend_title="Plant Type",
    template="plotly"
)

# Affichage interactif
fig.show()

In [114]:
df_total.to_csv("./data/21_01_2025_aloevera_with_2_branch.csv")